In [ ]:
!conda env create -f environment.yml -n uts-dl-torch

!conda activate uts-dl-torch


In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git /model-repo/YOLOX



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets


train_ds = datasets.CocoDetection(
    root="./dataset/Object_Detection/coco/train",
    annFile="./dataset/Object_Detection/coco/train/train_annotations.json",
)

vaild_ds = datasets.CocoDetection(
    root="./dataset/Object_Detection/coco/valid",
    annFile="./dataset/Object_Detection/coco/valid/valid_annotations.json",
)

test_ds = datasets.CocoDetection(
    root="./dataset/Object_Detection/coco/test",
    annFile="./dataset/Object_Detection/coco/test/test_annotations.json",
)



In [ ]:
# yolox = keras.applications.YOLOX